# 属性验证

## 使⽤ property 验证

In [49]:
def printholder(who):
    print(who.acct, who.name, who.age, who.remain, who.addr, sep=' / ')

def cardHolder_test(cls):
    CardHolder = cls
    bob = CardHolder('1234-5678', 'Bob Smith', 40, '123 main st')
    print('bob:', end=' ')
    printholder(bob)
    bob.name = 'Bob Q. Smith'
    bob.age  = 50
    bob.acct = '23-45-67-89'
    print('bob:', end=' ')
    printholder(bob)
    
    sue = CardHolder('5678-12-34', 'Sue Jones', 35, '124 main st')
    print('sue:', end=' ')
    printholder(sue)
    print('bob:', end=' ')
    printholder(bob)
    try:
        sue.age = 200
    except:
        print('Bad age for Sue')
    
    try:
        sue.remain = 5
    except:
        print("Can't set sue.remain")
    
    try:
        sue.acct = '1234567'
    except:
        print('Bad acct for Sue')


In [50]:
class CardHolder:                      # Need "(object)" for setter in 2.X
    acctlen = 8                                # Class data
    retireage = 59.5

    def __init__(self, acct, name, age, addr):
        self.acct = acct                       # Instance data
        self.name = name                       # These trigger prop setters too!
        self.age  = age                        # __X mangled to have class name
        self.addr = addr                       # addr is not managed
                                               # remain has no data
    def getName(self):
        return self.__name
    def setName(self, value):
        value = value.lower().replace(' ', '_')
        self.__name = value
    name = property(getName, setName)

    def getAge(self):
        return self.__age
    def setAge(self, value):
        if value < 0 or value > 150:
            raise ValueError('invalid age')
        else:
            self.__age = value
    age = property(getAge, setAge)

    def getAcct(self):
        return self.__acct[:-3] + '***'
    def setAcct(self, value):
        value = value.replace('-', '')
        if len(value) != self.acctlen:
            raise TypeError('invald acct number')
        else:
            self.__acct = value
    acct = property(getAcct, setAcct)

    def remainGet(self):                       # Could be a method, not attr
        return self.retireage - self.age       # Unless already using as attr
    remain = property(remainGet)


In [51]:
cardHolder_test(CardHolder)

bob: 12345*** / bob_smith / 40 / 19.5 / 123 main st
bob: 23456*** / bob_q._smith / 50 / 9.5 / 123 main st
sue: 56781*** / sue_jones / 35 / 24.5 / 124 main st
bob: 23456*** / bob_q._smith / 50 / 9.5 / 123 main st
Bad age for Sue
Can't set sue.remain
Bad acct for Sue


使⽤共享的描述符实例状态的验证

In [59]:
class CardHolder:                        # Need all "(object)" in 2.X only
    acctlen = 8                                  # Class data
    retireage = 59.5

    def __init__(self, acct, name, age, addr):
        self.acct = acct                         # Instance data
        self.name = name                         # These trigger __set__ calls too!
        self.age  = age                          # __X not needed: in descriptor
        self.addr = addr                         # addr is not managed
                                                 # remain has no data
    class Name(object):
        def __get__(self, instance, owner):      # Class names: CardHolder locals
            return self.name
        def __set__(self, instance, value):
            value = value.lower().replace(' ', '_')
            self.name = value
    name = Name()

    class Age(object):                                  
        def __get__(self, instance, owner):
            return self.age                             # Use descriptor data
        def __set__(self, instance, value):
            if value < 0 or value > 150:
                raise ValueError('invalid age')
            else:
                self.age = value
    age = Age()

    class Acct(object):                                 
        def __get__(self, instance, owner):
            return self.acct[:-3] + '***'
        def __set__(self, instance, value):
            value = value.replace('-', '')
            if len(value) != instance.acctlen:          # Use instance class data
                raise TypeError('invald acct number')
            else:
                self.acct = value
    acct = Acct()

    class Remain(object):                              
        def __get__(self, instance, owner):
            return instance.retireage - instance.age    # Triggers Age.__get__
        def __set__(self, instance, value):
            raise TypeError('cannot set remain')        # Else set allowed here
    remain = Remain()


In [60]:
cardHolder_test(CardHolder)

bob: 12345*** / bob_smith / 40 / 19.5 / 123 main st
bob: 23456*** / bob_q._smith / 50 / 9.5 / 123 main st
sue: 56781*** / sue_jones / 35 / 24.5 / 124 main st
bob: 56781*** / sue_jones / 35 / 24.5 / 123 main st
Bad age for Sue
Can't set sue.remain
Bad acct for Sue


In [61]:
CardHolder.name

'sue_jones'

使⽤基于客户实例状态的验证

In [63]:
class CardHolder:                        # Need all "(object)" in 2.X only
    acctlen = 8                                  # Class data
    retireage = 59.5

    def __init__(self, acct, name, age, addr):
        self.acct = acct                         # Client instance data
        self.name = name                         # These trigger __set__ calls too!
        self.age  = age                          # __X needed: in client instance
        self.addr = addr                         # addr is not managed
                                                 # remain managed but has no data
    class Name(object):
        def __get__(self, instance, owner):      # Class names: CardHolder locals
            return instance.__name
        def __set__(self, instance, value):
            value = value.lower().replace(' ', '_')
            instance.__name = value # 在 __set__ 方法中，它创建了一个名为 __name 的实例变量
    name = Name()                                       # class.name vs mangled attr

    class Age(object):
        def __get__(self, instance, owner):
            return instance.__age                       # Use descriptor data
        def __set__(self, instance, value):
            if value < 0 or value > 150:
                raise ValueError('invalid age')
            else:
                instance.__age = value
    age = Age()                                         # class.age vs mangled attr

    class Acct(object):
        def __get__(self, instance, owner):
            return instance.__acct[:-3] + '***'
        def __set__(self, instance, value):
            value = value.replace('-', '')
            if len(value) != instance.acctlen:          # Use instance class data
                raise TypeError('invald acct number')
            else:
                instance.__acct = value
    acct = Acct()                                       # class.acct vs mangled name

    class Remain(object):
        def __get__(self, instance, owner):
            return instance.retireage - instance.age    # Triggers Age.__get__
        def __set__(self, instance, value):
            raise TypeError('cannot set remain')        # Else set allowed here
    remain = Remain()


In [64]:
cardHolder_test(CardHolder)

bob: 12345*** / bob_smith / 40 / 19.5 / 123 main st
bob: 23456*** / bob_q._smith / 50 / 9.5 / 123 main st
sue: 56781*** / sue_jones / 35 / 24.5 / 124 main st
bob: 23456*** / bob_q._smith / 50 / 9.5 / 123 main st
Bad age for Sue
Can't set sue.remain
Bad acct for Sue


不支持类访问

In [65]:
CardHolder.name

AttributeError: 'NoneType' object has no attribute '_Name__name'

使⽤. _getattr__ 验证

In [128]:
class CardHolder:
    acctlen = 8                                  # Class data
    retireage = 59.5

    def __init__(self, acct, name, age, addr):
        self.acct = acct                         # Instance data
        self.name = name                         # These trigger __setattr__ too
        self.age  = age                          # _acct not mangled: name tested
        self.addr = addr                         # addr is not managed
                                                 # remain has no data
    def __getattr__(self, name):
        print('__getattr__', name)
        if name == 'acct':                           # On undefined attr fetches，调用 __getattr__
            # return self._acct[:-3] + '***'           # name, age, addr are defined
            return object.__getattribute__(self, '_acct')[:-3] + '***'           # name, age, addr are defined
        elif name == 'remain':
            return self.retireage - self.age         # Doesn't trigger __getattr__
        else:
            raise AttributeError(name)

    def __setattr__(self, name, value):
        if name == 'name':                           # On all attr assignments
            value = value.lower().replace(' ', '_')  # addr stored directly
        elif name == 'age':                          # acct mangled to _acct
            if value < 0 or value > 150:
                raise ValueError('invalid age')
        elif name == 'acct':
            print('__setattr__', name)
            name  = '_acct' # 随后对 acct 的访问还会调⽤ __getattr__
            value = value.replace('-', '')
            if len(value) != self.acctlen:
                raise TypeError('invald acct number')
        elif name == 'remain': # 未定义属性，会调用 __getattr__
            raise TypeError('cannot set remain')
        # self.__dict__[name] = value                  # Avoid looping (or via object)
        object.__setattr__(self, name, value)


In [129]:
cardHolder_test(CardHolder)

__setattr__ acct
bob: __getattr__ acct
__getattr__ remain
12345*** / bob_smith / 40 / 19.5 / 123 main st
__setattr__ acct
bob: __getattr__ acct
__getattr__ remain
23456*** / bob_q._smith / 50 / 9.5 / 123 main st
__setattr__ acct
sue: __getattr__ acct
__getattr__ remain
56781*** / sue_jones / 35 / 24.5 / 124 main st
bob: __getattr__ acct
__getattr__ remain
23456*** / bob_q._smith / 50 / 9.5 / 123 main st
Bad age for Sue
Can't set sue.remain
__setattr__ acct
Bad acct for Sue


In [118]:
bob = CardHolder('1234-1456', 'Bob', 50, '123 main st')
bob._acct, bob.acct

__setattr__ acct
__getattr__ acct


('12341456', '12341***')

In [121]:
[x for x in dir(bob) if not x.startswith('__')]

['_acct', 'acctlen', 'addr', 'age', 'name', 'retireage']

In [123]:
for x in (v for v in dir(bob) if not v.startswith('__')):
    print(f'{x:>10}=> {getattr(bob, x)}')

     _acct=> 12341456
   acctlen=> 8
      addr=> 123 main st
       age=> 50
      name=> bob
 retireage=> 59.5


使⽤__getattribute__ 验证

In [87]:
class CardHolder(object):                        # Need "(object)" in 2.X only
    acctlen = 8                                  # Class data
    retireage = 59.5

    def __init__(self, acct, name, age, addr):
        self.acct = acct                         # Instance data
        self.name = name                         # These trigger __setattr__ too
        self.age  = age                          # acct not mangled: name tested
        self.addr = addr                         # addr is not managed
                                                 # remain has no data
    def __getattribute__(self, name):
        print('__getattribute__', name)
        superget = object.__getattribute__             # Don't loop: one level up
        if name == 'acct':                             # On all attr fetches
            return superget(self, 'acct')[:-3] + '***'
        elif name == 'remain':
            return superget(self, 'retireage') - superget(self, 'age')
        else:
            return superget(self, name)                # name, age, addr: stored

    def __setattr__(self, name, value):
        print('__setattr__', name)
        if name == 'name':                             # On all attr assignments
            value = value.lower().replace(' ', '_')    # addr stored directly
        elif name == 'age':
            if value < 0 or value > 150:
                raise ValueError('invalid age')
        elif name == 'acct':
            value = value.replace('-', '')
            if len(value) != self.acctlen:
                raise TypeError('invald acct number')
        elif name == 'remain':
            raise TypeError('cannot set remain')
        # self.__dict__[name] = value                     # Avoid loops, orig names
        object.__setattr__(self, name, value)
        

In [88]:
cardHolder_test(CardHolder)

__setattr__ acct
__getattribute__ acctlen
__setattr__ name
__setattr__ age
__setattr__ addr
bob: __getattribute__ acct
__getattribute__ name
__getattribute__ age
__getattribute__ remain
__getattribute__ addr
12345*** / bob_smith / 40 / 19.5 / 123 main st
__setattr__ name
__setattr__ age
__setattr__ acct
__getattribute__ acctlen
bob: __getattribute__ acct
__getattribute__ name
__getattribute__ age
__getattribute__ remain
__getattribute__ addr
23456*** / bob_q._smith / 50 / 9.5 / 123 main st
__setattr__ acct
__getattribute__ acctlen
__setattr__ name
__setattr__ age
__setattr__ addr
sue: __getattribute__ acct
__getattribute__ name
__getattribute__ age
__getattribute__ remain
__getattribute__ addr
56781*** / sue_jones / 35 / 24.5 / 124 main st
bob: __getattribute__ acct
__getattribute__ name
__getattribute__ age
__getattribute__ remain
__getattribute__ addr
23456*** / bob_q._smith / 50 / 9.5 / 123 main st
__setattr__ age
Bad age for Sue
__setattr__ remain
Can't set sue.remain
__setattr__ a